In [1]:
# !pip install transformers==2.11.0
# !pip install tensorboardx
# !pip install simpletransformers==0.41.1

In [2]:
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel

In [3]:
df = pd.read_csv('clean_datav4.csv')
# df

In [4]:
df

,Unnamed: 0,Unnamed: 0.1,ID,tweet_id,text,missing_text,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose
0,0,0,0,1.052240e+18,new post domestic violence awareness caught me...,0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1,1,1,1.052210e+18,domestic violence awareness caught metoo,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,2,2,1.052180e+18,mother nature metoo,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,3,1.052160e+18,ption no2,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,4,4,1.052100e+18,high time metoo named shamed men medium advert...,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7973,7973,7973,7973,1.052100e+18,one priyaramani make billion people metooindia...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7974,7974,7974,7974,1.052100e+18,thought metoo limited woman condeming wake rea...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7975,7975,7975,7975,1.052100e+18,wake metoo movement hairstylist sapna bhavani ...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7976,7976,7976,7976,1.052100e+18,metoo icc step sexual harassment,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
temp = df[['Support', 'Oppose']].values.sum(axis=1)
temp[temp==2]

array([], dtype=float64)

**Note:** This implies that there are tweets which are in either support or oppose or in neither. But there are no tweets that are classified as support and oppose 

In [6]:
text = df['text'].values.reshape(-1, 1)

In [7]:
rows = text.shape[0]
rows

7978

In [8]:
support = df['Support'].values.reshape(-1, 1)
oppose = df['Oppose'].values.reshape(-1, 1)

In [9]:
labels = np.zeros(support.shape)

**Labels are:**
0 - Neutral; 
1 - Support; 
2 - Oppose; 

In [10]:
labels = support * 1 + oppose * 2
twos = labels[labels==2]
ones = labels[labels==1]

In [11]:
print(ones.shape, twos.shape)

(2501,) (602,)


In [12]:
data_arr = np.concatenate((text, labels), axis=1)
data_twos=data_arr[data_arr[:, 1]==2]
data_ones=data_arr[data_arr[:, 1]==1]
data_zeros=data_arr[data_arr[:, 1]==0]

In [13]:
print(data_zeros.shape, data_ones.shape, data_twos.shape)

(4875, 2) (2501, 2) (602, 2)


In [14]:
rows * 0.75

5983.5

In [15]:
train_rows = 5983 #0.75
test_rows = 7978 - 5983 #0.25

In [16]:
from sklearn.model_selection import train_test_split
ones_text_train, ones_text_test, ones_label_train, ones_label_test = train_test_split(data_ones[:, 0], data_ones[:, 1], train_size=0.9, shuffle=True)

In [17]:
ones_text_test.shape

(251,)

In [18]:
twos_text_train, twos_text_test, twos_label_train, twos_label_test = train_test_split(data_twos[:, 0], data_twos[:, 1], train_size=0.9, shuffle=True)

In [19]:
twos_text_test.shape

(61,)

In [20]:
ones_text_train.shape[0]+twos_text_train.shape[0]

2791

In [26]:
req_zeros = train_rows - (ones_text_train.shape[0]+twos_text_train.shape[0])
req_zeros

3192

In [22]:
np.random.shuffle(data_zeros)

In [23]:
data_zeros

array([['october metoo hashtag start trending twitter october kavanaugh installed u supreme court got work crowdfunding anatomy silence ',
        0.0],
       ['metoo somebody tried rape say alia bhatt mother soni razdan ',
        0.0],
       ['metoo john abraham come forward amp sack casting agent vicky sidana batla house credit roll ',
        0.0],
       ...,
       ['breaking anu malik fired judge indian idol season following multiple sexual harassment allegation metoo metooindia timesup sunilpa44775222 rubikaliyaquat sonalchauhan7 ',
        0.0],
       ['fung supporter ginaraimondo support choice metoo amp lgbtq right vote fung ',
        0.0],
       ['people live chronicpain like fibromyalgia often disbelieved need metoo movement time end ridiculous debate whether fibro real find ',
        0.0]], dtype=object)

In [24]:
zeros_text_train = data_zeros[:req_zeros, 0]
zeros_text_test = data_zeros[req_zeros:, 0]

In [25]:
zeros_label_train = data_zeros[:req_zeros, 1]
zeros_label_test = data_zeros[req_zeros:, 1]

In [27]:
text_train = np.concatenate((zeros_text_train, ones_text_train, twos_text_train), axis=0)

In [28]:
text_train.shape

(5983,)

In [29]:
label_train = np.concatenate((zeros_label_train, ones_label_train, twos_label_train), axis=0)
label_train.shape

(5983,)

In [30]:
text_test = np.concatenate((zeros_text_test, ones_text_test, twos_text_test), axis=0)

In [31]:
label_test = np.concatenate((zeros_label_test, ones_label_test, twos_label_test), axis=0)


In [34]:
text_train=text_train.reshape(-1, 1)
text_test=text_test.reshape(-1, 1)
label_train=label_train.reshape(-1, 1)
label_test=label_test.reshape(-1, 1)

In [38]:
train_df = pd.DataFrame(data=np.concatenate((text_train, label_train), axis=1), columns=['text', 'labels'], index=None).sample(frac=1)
test_df = pd.DataFrame(data=np.concatenate((text_test, label_test), axis=1), columns=['text', 'labels'], index=None).sample(frac=1)

In [39]:
np.unique(train_df['labels'], return_counts=True)

(array([0.0, 1.0, 2.0], dtype=object), array([3192, 2250,  541]))

In [40]:
np.unique(test_df['labels'], return_counts=True)


(array([0.0, 1.0, 2.0], dtype=object), array([1683,  251,   61]))

In [41]:
test_df

,text,labels
931,hear kate palmer associate director advice pen...,0
341,irks number people supporting accepting encour...,0
309,metoo elclassico,0
1596,salmankhan remains silent metoo father salimkh...,0
871,metoo alyssamilano kathygriffin woman truthful,0
...,...,...
1719,hey metoo metooindia mentoo wetoo himtoo agree...,1
1287,ption no2,0
1478,news update metoo sc decline urgent hearing pi...,0
230,tonirambold gziele prepared,0


In [44]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=3, use_cuda=False)

Downloading: 100%|██████████| 501M/501M [00:46<00:00, 10.8MB/s]    
Downloading: 100%|██████████| 899k/899k [00:01<00:00, 660kB/s]  
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 468kB/s]  


In [45]:
model.train_model(train_df)

Running Epoch 0:   0%|          | 0/748 [00:00<?, ?it/s]

Running loss: 1.097642


Running Epoch 0:   0%|          | 1/748 [00:05<1:05:18,  5.25s/it]

Running loss: 1.106650


Running Epoch 0:   0%|          | 2/748 [00:13<1:17:57,  6.27s/it]

...


Running Epoch 0:   2%|▏         | 16/748 [01:17<55:11,  4.52s/it]

Running loss: 1.062869


Running Epoch 0:   2%|▏         | 17/748 [01:21<53:51,  4.42s/it]

Running loss: 1.040105

KeyboardInterrupt: 